# The Cozo database tutorial

This tutorial will teach you the basics of using the Cozo database with the query language CozoScript.
There are no database-specific prerequisites, 
though it would be helpful if you already know some other databases, 
especially SQL databases.

## Setup

The best way to learn from this tutorial is to run the queries as they are introduced.
For this, you need to install the Cozo database on your local machine.

Cozo is distributed as a single gzipped/zipped binary. 
Go to https://github.com/cozodb/cozo/releases and download the latest release binary for your operating system to a local directory.

If your operating system is Linux/Mac, 
open a terminal command prompt, 
`cd` into the directory of your download,
and run

If you are on Windows instead, open a PowerShell and run

To run the server, you need to specify a directory to store persistent data on your file system. 
In the following we will use a directory called `tutorial-data` in the same directory as the binary executable.
In the terminal, run

The same command should work in powershell as well.

If you see something like `Database web API running at ...` displayed in your terminal, 
then the server is successfully started. 
Keep the server running when you are following the tutorial.
When you are done, `CTRL-C` in the terminal will stop the server.
You can restart the server again by running the command again.

More options when starting the server is available. Run

for more details.

## A place to run queries

Cozo exposes an HTTP API, so theoretically you can follow along using tools like `curl`. 
If you are interested, consult the manual for the request format the API expects.
For better user experience, though, we suggest following one of the following two subsections instead.

### Option 1: the JupyterLab notebook

This option provides the best user experience but also requires you to install quite a lot of things, 
though you may already have them installed on your computer if you use the python data science stack.
First you will need python installed. 
Then install JupyterLab by following the instruction at https://jupyter.org/install.
Next, run the following to install a Jupyter extension to help querying Cozo:

While you are at it, go to the source of this tutorial at https://github.com/cozodb/cozo/blob/main/tutorial/tutorial.ipynb, 
right-click on `Raw` and save the tutorial document to your disk.

Then run jupyter lab, open a the saved tutorial document, and follow along.

We need to enable the extension in the notebook. Run

In [1]:
%load_ext pycozo.ipyext_direct

Then the "hello world" query:

In [6]:
?[] <- [['hello', 'world', 'Cozo!']]

,0,1,2
0,hello,world,Cozo!


If you get the same words back formatted in a table, congratulations! 
You can skip to the next section where we starts learning CozoScript proper.
If you want to know more about what the `pycozo` extension did, read the manual.

### Option 2: the JavaScript console in your browser

If you have never used Python before, the first option may be overwhelming. 
Or you just want to try Cozo out first to decide quickly if it has anything interesting for you.
Whatever your reason for not wanting to install the whole python toolchain, we have you covered.

Your local machine at least have a modern web browser, like a recent version of Firefox, Chrome, or Edge, right?
Good. 

Use your browser to navigate to http://127.0.0.1:9070 (the address shown in your terminal when you run `cozoserver`).
You should be greeted be a page saying that the server is running.
Now open the developer tools of your browser by right clicking the page and select "Inspect" from the menu 
(if you cannot find it, you may need to fiddle with your browser settings to enable the developer tools).
Switch to the "Console" tab of the developer tools if it is not already open. 

If you see some messages where 
the "Cozo Makeshift Javascript Console" welcomes you, you are ready. Run the "hello world" query by typing the following into the console and press enter:

If you see the three words echoed back in a table, you are successful. When following the tutorial, you have to wrap all queries within the backticks `` in the above command to run them in the JavaScript console.

## Rules and relations

Cozo is a relational database. As you might have guessed, the "hello world" query

In [7]:
?[] <- [['hello', 'world', 'Cozo!']]

,0,1,2
0,hello,world,Cozo!


simply passes an ad hoc relation, here represented by a list of lists, and ask the database to return the relation to you.
You can pass more rows, or a different number of columns, to corroborate further your guess:

In [8]:
?[] <- [[1, 2, 3], ['a', 'b', 'c']]

,0,1,2
0,1,2,3
1,a,b,c


In [15]:
?[] <- [[1.5, 2.5, 3, 4, 5.5], 
        ['aA', 'bB', 'cC', 'dD', 'eE'], 
        [true, false, null, -1.4e-2, "A string with double quotes"]]

,0,1,2,3,4
0,True,False,None,-0.014000,A string with double quotes
1,1.500000,2.500000,3,4,5.500000
2,aA,bB,cC,dD,eE


The last example shows how to enter literals for numbers, strings, booleans and `null`.
The literal representations are similar to those in JavaScript. 
In particular, strings in double quotes are guaranteed to be interpreted in exactly the same way as in JSON.

You may be surprised by the order of the returned rows in the last example: the returned order is not the same as the input order.
This is because in Cozo relations are stored (either in memory or on disk) as trees, and trees are always sorted.
Another consequence of trees is that you can have no duplicate rows:

In [14]:
?[] <- [[1], [2], [1], [2], [1]]

,0
0,1
1,2


We say that relations in Cozo follow _set semantics_ where de-duplication is automatic. 
By constrast, SQL usually follows _bag semantics_ (some databases does this by secretly having a unique internal key for every row, in Cozo you must do this explicitly if you need to simulate duplicate rows).
Why does Cozo break tradition and go with set semantics?
Set semantics is much more convenient when you have recursions between relations involved,
and Cozo is designed to deal with very complicated recursions.